<a href="https://colab.research.google.com/github/KathyRoma/DS-Unit-1-Sprint-2-Statistics/blob/master/DS-Unit-1-Sprint-2-Statistics/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

In [0]:
### YOUR CODE STARTS HERE

In [0]:
# I start with importing libraries
import pandas as pd
import numpy as np


In [1]:
# I download the dataset using shell command
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2020-05-11 21:10:16--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2020-05-11 21:10:17 (278 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [4]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data')
df.head()

,republican,n,y,n.1,y.1,y.2,y.3,n.2,n.3,n.4,y.4,?,y.5,y.6,y.7,n.5,y.8
0,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
1,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
2,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
3,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
4,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y


In [5]:
# I use the description from the source to define appropriate column names and then create a header

column_headers = ['party', 'handicapped-infants', 'water-project', 
                  'budget', 'physician-fee-freeze', 'el-salvador-aid', 
                  'religious-groups', 'anti-satellite-ban', 'aid-to-contras', 
                  'mx-missile', 'immigration', 'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free', 'south-africa']

df= pd.read_csv('house-votes-84.data', names = column_headers)

df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [7]:
# Let's have a look at our data
df.shape

(435, 17)

In [8]:
# Common sense tells us that the first column should consist of two values -- parties. 
## And yes, our assumption is correct
df['party'].value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [9]:
# I perform some categorical encoding to replace string values 'y' and 'n' with numeric equivalents, and replace '?' with a Nan
df = df.replace({'y' : 1, 'n' : 0, '?' : np.NaN})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# Before testing amy hypothesies I divide the dataframe into two, making two party-homogeneous sets
dem = df[df['party'] == 'democrat']
rep = df[df['party'] == 'republican']

In [0]:
# I need another library to perform testing
from scipy import stats

In [13]:
# Let's have a closer look at the 'relegious-groups' bill. 
# I suggest the bill content might be devisive for republicans and democrats.
## First I count the column means to find out the number of members supported the bill
print(rep['religious-groups'].mean())
print(dem['religious-groups'].mean())

## Just from these two figures we can say that republicans show greater support to the bill 
#(especially given their smaller number in the house of reps)

0.8975903614457831
0.47674418604651164


Testing parameters

1) My **Null Hypothesis (NH) is:** the support for 'relegious groups' bill is equal from both republicans and democrats

Sample means of the two samples should be equal

2) **Alternative Hypothesis (AH)**: Support is different

3) Confidence Level (CL): **99%**


*What I want to find out is who's support is greater and to what extent*

In [14]:
# Now I perform the test
# Wow, my pvalue is really small,  and I'm affraid we have to reject our NH
## Also, statistics is negative, it proves greater support from republicans
stats.ttest_ind(dem['religious-groups'], rep['religious-groups'], nan_policy='omit')

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

In [15]:
# let's test another bill with the same NH, AH and CL
## We get pvalue = 0.08 > 0.01, and we fail to regect our NH
stats.ttest_ind(dem['immigration'], rep['immigration'], nan_policy='omit')

Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

In [16]:
# I want to check the means to be sure
## Not so different indeed. 
print(dem['immigration'].mean())
print(rep['immigration'].mean())

0.4714828897338403
0.5575757575757576


In [0]:
# What if I don't want to test every bill manually, but rather get a complete picture of support in one place?
## I create a new dataframe with columns showing each bill ttest results and also mean values for democrats and republicans

In [17]:
ttests_results = pd.DataFrame(columns = ['bills', 'stats','pval', 'dem_mean', 'rep_mean'])
for x in column_headers[1:]:
  a = stats.ttest_ind(dem[x], rep[x], nan_policy='omit')
  print(a)
  print()
  print(rep[x].mean())
  ttests_results = ttests_results.append({'bills':x, 'stats':a.statistic, 
                         'pval': a.pvalue, 'dem_mean':dem[x].mean(), 
                         'rep_mean':rep[x].mean()}, ignore_index=True)


 

Ttest_indResult(statistic=9.205264294809222, pvalue=1.613440327937243e-18)

0.18787878787878787
Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

0.5067567567567568
Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

0.13414634146341464
Ttest_indResult(statistic=-49.36708157301406, pvalue=1.994262314074344e-177)

0.9878787878787879
Ttest_indResult(statistic=-21.13669261173219, pvalue=5.600520111729011e-68)

0.9515151515151515
Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

0.8975903614457831
Ttest_indResult(statistic=12.526187929077842, pvalue=8.521033017443867e-31)

0.24074074074074073
Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

0.15286624203821655
Ttest_indResult(statistic=16.437503268542994, pvalue=5.03079265310811e-47)

0.11515151515151516
Ttest_indResult(statistic=-1.7359117329695164, pvalue=0.08330248490425066)

0.5575757575757576
Ttest_indResult(statistic=8.2936039894

In [18]:
# And I want it to look beautiful here
ttests_results

,bills,stats,pval,dem_mean,rep_mean
0,handicapped-infants,9.205264,1.613440e-18,0.604651,0.187879
1,water-project,-0.088965,9.291557e-01,0.502092,0.506757
2,budget,23.212777,2.070340e-77,0.888462,0.134146
3,physician-fee-freeze,-49.367082,1.994262e-177,0.054054,0.987879
4,el-salvador-aid,-21.136693,5.600520e-68,0.215686,0.951515
5,religious-groups,-9.737576,2.393672e-20,0.476744,0.897590
6,anti-satellite-ban,12.526188,8.521033e-31,0.772201,0.240741
7,aid-to-contras,18.052093,2.824718e-54,0.828897,0.152866
8,mx-missile,16.437503,5.030793e-47,0.758065,0.115152
9,immigration,-1.735912,8.330248e-02,0.471483,0.557576


In [35]:
#Let's sort this new df in search for the least controversial bills
## I create a condition to check: 'waterproject' and 'immigration'(what a surprise!) are equally supported bills
pvals = ttests_results['pval']>0.01
ttests_results[pvals].head()

,bills,stats,pval,dem_mean,rep_mean
1,water-project,-0.088965,0.929156,0.502092,0.506757
9,immigration,-1.735912,0.083302,0.471483,0.557576


In [0]:
# As long as I put democrats first in my test, all positive statistic values show us 
# that democratic support of that particular bill prevails 

In [33]:
#I write another condition to find out positive stats values and at the same time filtering it by p value
stats_dem = (ttests_results['stats']>0) & (ttests_results['pval']<0.01)
stats_dem

0      True
1     False
2      True
3     False
4     False
5     False
6      True
7      True
8      True
9     False
10     True
11    False
12    False
13    False
14     True
15     True
dtype: bool

In [34]:
# These bills are supported by democrats in a larger extent then by republicans 
ttests_results[stats_dem].head()

,bills,stats,pval,dem_mean,rep_mean
0,handicapped-infants,9.205264,1.613440e-18,0.604651,0.187879
2,budget,23.212777,2.070340e-77,0.888462,0.134146
6,anti-satellite-ban,12.526188,8.521033e-31,0.772201,0.240741
7,aid-to-contras,18.052093,2.824718e-54,0.828897,0.152866
8,mx-missile,16.437503,5.030793e-47,0.758065,0.115152


In [32]:
# I am going to do the same for republican support;
# now we search for negative stats and p value<0.01
stats_rep = (ttests_results['stats']<0) & (ttests_results['pval']<0.01)
ttests_results[stats_rep].head()

,bills,stats,pval,dem_mean,rep_mean
3,physician-fee-freeze,-49.367082,1.994262e-177,0.054054,0.987879
4,el-salvador-aid,-21.136693,5.600520e-68,0.215686,0.951515
5,religious-groups,-9.737576,2.393672e-20,0.476744,0.897590
11,education,-20.500686,1.883420e-64,0.144578,0.870968
12,right-to-sue,-13.510643,1.227858e-34,0.289683,0.860759


## Stretch Goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Work on Performing a T-test without using Scipy in order to get "under the hood" and learn more thoroughly about this topic.
### Start with a 1-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://blog.minitab.com/hs-fs/hubfs/Imported_Blog_Media/701f9c0efa98a38fb397f3c3ec459b66.png?width=247&height=172&name=701f9c0efa98a38fb397f3c3ec459b66.png) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then try a 2-sample t-test
 - Establish the conditions for your test 
 - [Calculate the T Statistic](https://lh3.googleusercontent.com/proxy/rJJ5ZOL9ZDvKOOeBihXoZDgfk7uv1YsRzSQ1Tc10RX-r2HrRpRLVqlE9CWX23csYQXcTniFwlBg3H-qR8MKJPBGnjwndqlhDX3JxoDE5Yg) (You'll need to omit NaN values from your sample).
 - Translate that t-statistic into a P-value. You can use a [table](https://www.google.com/search?q=t+statistic+table) or the [University of Iowa Applet](https://homepage.divms.uiowa.edu/~mbognar/applets/t.html)

 ### Then check your Answers using Scipy!